# Custom Model Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to create and deploy custom model which using IRIS classifier based on xgboost model into Merlin.

In [ ]:
import merlin
import warnings
import os
import xgboost as xgb
from merlin.model import ModelType
from sklearn.datasets import load_iris
warnings.filterwarnings('ignore')
print(merlin.__version__)

## 1. Initialize Merlin Resources


### 1.1 Set Merlin Server

In [ ]:
# Set MLP Server
MERLIN_SERVER_URL = os.environ.get("MERLIN_SERVER_URL", "localhost:8080/api/merlin")
merlin.set_url(MERLIN_SERVER_URL)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("custom-model", ModelType.CUSTOM)

## 2. Train and Deploy Images

### 2.1 Train Model

In [ ]:
model_dir = "xgboost-model"
BST_FILE = "model.bst"

iris = load_iris()
y = iris['target']
X = iris['data']
dtrain = xgb.DMatrix(X, label=y)
param = {'max_depth': 6,
            'eta': 0.1,
            'silent': 1,
            'nthread': 4,
            'num_class': 10,
            'objective': 'multi:softmax'
            }
xgb_model = xgb.train(params=param, dtrain=dtrain)
model_file = os.path.join((model_dir), BST_FILE)
xgb_model.save_model(model_file)

### 2.2 Create Model Version and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_custom_model(image="ghcr.io/gojek/custom-model:v0.3",model_dir=model_dir)
v.finish()
```


This image `afif2100/caraml-dev-merlin-sample-test-custom-model:latest` is built by using this [Dockerfile](./server/dockerfile). The image contains python fast-api web service executable where the code you can find [here](./server)

In [ ]:
# Create new version of the model
with merlin.new_model_version() as v:
    # Upload the serialized model to Merlin
    merlin.log_custom_model(image="afif2100/caraml-dev-merlin-sample-test-custom-model:latest", model_dir=model_dir)
    

### 2.2 Deploy Model


In [ ]:
from merlin.protocol import Protocol

endpoint = merlin.deploy(v, protocol = Protocol.HTTP_JSON)

### 2.3 Send Test Request

In [ ]:
# Test deployment
import requests

# Get endpoint
data = {"instances": [[1,2,3,4], [2,1,2,4]]}

# Send request
response = requests.post(endpoint.url, json=data)
print(response.json())

In [ ]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -H "Content-Type: application/json" -d '{"instances": [[1,2,3,4], [2,1,2,4]]}'

### 2.4 Delete Deployment

In [ ]:
merlin.undeploy(v)